In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load models
classification_model = tf.keras.models.load_model('/content/drive/MyDrive/classification_model.h5')
segmentation_model = tf.keras.models.load_model('/content/drive/MyDrive/segmentation_model.h5')

classification_model.summary()

# Function for preprocessing the image
def preprocess_image(image, target_size):
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)
    image = image.resize(target_size)
    image_array = np.array(image) / 255.0
    if len(image_array.shape) == 2:
        image_array = np.stack((image_array,) * 3, axis=-1)
    return np.expand_dims(image_array, axis=0)

# Function for classification
def classify_image(image):
    processed_image = preprocess_image(image, target_size=(150, 150))
    prediction = classification_model.predict(processed_image)
    return "Tumor detected" if prediction[0][0] > 0.5 else "No tumor"

# Function for segmentation with custom color for tumor
def segment_tumor(image, tumor_color=(255, 0, 0)):
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    original_size = image.size
    processed_image = preprocess_image(image, target_size=(128, 128))

    # Perform segmentation prediction
    segmentation_map = segmentation_model.predict(processed_image)

    # Threshold the output and scale to 0-255
    segmentation_map = (segmentation_map[0, :, :, 0] > 0.5).astype(np.uint8) * 255

    # Resize the segmentation map to match the original image size
    segmentation_map_resized = Image.fromarray(segmentation_map).resize(original_size, Image.NEAREST)

    # Create an overlay image to combine the original image with the segmentation mask
    image_array = np.array(image)
    if len(image_array.shape) == 2:
        image_array = np.stack((image_array,) * 3, axis=-1)  # Convert grayscale to RGB

    segmentation_map_resized = np.array(segmentation_map_resized)

    # Create a colored segmentation map (red by default)
    segmentation_map_colored = np.zeros_like(image_array, dtype=np.uint8)
    segmentation_map_colored[segmentation_map_resized == 255] = tumor_color  # Color the tumor area

    # Combine the original image and the colored tumor mask with transparency
    overlay = Image.blend(Image.fromarray(image_array), Image.fromarray(segmentation_map_colored), alpha=0.5)
    return overlay

# Complete pipeline
def analyze_brain(image):
    if image is None:
        return "No image provided", None

    classification_result = classify_image(image)
    if classification_result == "Tumor detected":
        segmentation_result = segment_tumor(image)
        return classification_result, segmentation_result
    else:
        return classification_result, None

# Gradio interface
def interface():
    inputs = gr.Image(type="numpy", label="Upload Brain MRI Image")
    outputs = [
        gr.Textbox(label="Classification Result"),
        gr.Image(type="pil", label="Segmentation Map")
    ]
    gr.Interface(
        fn=analyze_brain,
        inputs=inputs,
        outputs=outputs,
        title="Brain MRI Tumor Analysis"
    ).launch(debug=True)

if __name__ == "__main__":
    interface()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,034,179 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a37a42c9508b090606.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 494ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
